In [3]:
import torch
import sys
import pathlib
import json
import re
import omegaconf
import tqdm
import git
import concurrent.futures as cf
import queue

git_root = pathlib.Path(git.Repo(search_parent_directories=True).git_dir).parent
target = git_root / "approach_sft"
assert target.exists(), f"Target directory does not exist: {target}"

sys.path.append(str(target))

In [6]:
ROOT_PATH = pathlib.Path("~/scratch/marglicot_saves/sft_saves/0_001/").expanduser().resolve()
paths = list(ROOT_PATH.glob("*gsm8k*/*/meta_info.pt"))
assert paths, "No paths found"



def json_default(obj):
    if isinstance(obj, pathlib.Path):
        return str(obj)
    elif isinstance(obj, torch.Tensor):
        print(obj.dtype, obj.shape)
    if isinstance(obj, torch.dtype):
        return str(obj)
    raise TypeError(f"Object of type {type(obj)} is not JSON serializable")


def process_file(input_path):
    target_path = input_path.parent / re.sub(r"\.pt$", ".json", input_path.name)
    with open(input_path, "rb") as f:
        data = torch.load(f, map_location="cpu")
    
    data["cfg"] = omegaconf.OmegaConf.to_container(data["cfg"], resolve=True)
    del data["optimizer"]

    with open(target_path, "w") as f:
        json.dump(data, f, indent=4, sort_keys=True, default=json_default)
    return input_path

with cf.ThreadPoolExecutor(5) as executor:
    futures = [executor.submit(process_file, path) for path in paths]
    for future in tqdm.tqdm(cf.as_completed(futures), total=len(futures)):
        try:
            future.result()
        except Exception as e:
            print(f"Error processing file: {e}")


/tmp/ipykernel_3391660/3738313570.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f, map_location="cpu")
  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:23<00:00,  2.64s/it]
